In [1]:
import pandas as pd
from nltk import word_tokenize
import nltk
from nltk.util import ngrams
from collections import Counter
import numpy as np
from fuzzywuzzy import fuzz
# pd.set_option('display.max_columns', 100)
# pd.set_option('display.max_colwidth', 100000)
# pd.set_option('display.max_rows', 500)
import string

In [2]:
%reload_kedro

2020-01-28 23:13:33,486 - root - INFO - ** Kedro project sneakairs
2020-01-28 23:13:33,487 - root - INFO - Defined global variable context


In [3]:
df = context.io.load('KOF_04152019_05')

2020-01-28 23:13:33,515 - kedro.io.data_catalog - INFO - Loading data from `KOF_04152019_05` (CSVLocalDataSet)...


In [4]:
df.head(1)

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,silhouette,...,sb,ls,nrg,prm,nsw,retro,se,pe,gs,hs
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,Air Foamposite One,...,0,0,0,0,0,0,0,0,0,0


In [5]:
len(df)

6338

## TF-IDF on sneaker names

In [6]:
names = list(df.name)

In [7]:
len(names)

6338

In [8]:
def tuple2string(tup): 
    str =  ' '.join(tup) 
    return str

In [9]:
grams = {}
for i in range(len(names)):
    all_grams = []
    for j in range(3,7):
        shoe = names[i]
        shoe = shoe.replace('nike ','')
        shoe = shoe.replace(' x ', '')
        shoe = shoe.replace('adidas', '')
        shoe = shoe.replace('og', '')
        shoe = shoe.replace('wmns', '')
        shoe = shoe.replace('high', '')
        shoe = shoe.replace('low', '')
        shoe = shoe.replace('gs', '')
        shoe = shoe.replace('mid', '')
        shoe = shoe.replace('retro', '')
        shoe = shoe.replace('footwear', '')
        shoe = shoe.replace('black', '')
        shoe = shoe.replace('white', '')
        shoe = shoe.translate(str.maketrans('','',string.punctuation))
        
        
        ngrams_raw = list(ngrams(shoe.split(), j))
        ngrams_clean = [tuple2string(ngrams_raw[i]) for i in range(len(ngrams_raw))]
        all_grams.extend(ngrams_clean)
    grams[names[i]] = all_grams

In [10]:
score_grams = []
for keys in grams:
    for j in range(len(grams[keys])):
        score = fuzz.ratio(keys, grams[keys][j])
        score_grams.append([keys, grams[keys][j], score])

In [11]:
silhouette = pd.DataFrame(score_grams, columns = ['name', 'ngram', 'score'])

In [12]:
silhouette

,name,ngram,score
0,Nike Air Foamposite One Snakeskin,Nike Air Foamposite,73
1,Nike Air Foamposite One Snakeskin,Air Foamposite One,71
2,Nike Air Foamposite One Snakeskin,Foamposite One Snakeskin,84
3,Nike Air Foamposite One Snakeskin,Nike Air Foamposite One,82
4,Nike Air Foamposite One Snakeskin,Air Foamposite One Snakeskin,92
...,...,...,...
61808,Air Jordan 11 Concord OG (1995),Jordan 11 Concord OG,78
61809,Air Jordan 11 Concord OG (1995),11 Concord OG 1995,73
61810,Air Jordan 11 Concord OG (1995),Air Jordan 11 Concord OG,87
61811,Air Jordan 11 Concord OG (1995),Jordan 11 Concord OG 1995,89


In [13]:
avg_ngram_scores = silhouette.groupby(['ngram'])['score'].mean().reset_index(name='avg_score')
num_ngrams = silhouette.groupby(['ngram'])['score'].count().reset_index(name='total')

In [14]:
silhouette = pd.merge(silhouette, num_ngrams, how='left', on='ngram')

In [15]:
silhouette = pd.merge(silhouette, avg_ngram_scores, how='left', on='ngram')

In [16]:
silhouette.head(500)

,name,ngram,score,total,avg_score
0,Nike Air Foamposite One Snakeskin,Nike Air Foamposite,73,59,69.711864
1,Nike Air Foamposite One Snakeskin,Air Foamposite One,71,38,67.394737
2,Nike Air Foamposite One Snakeskin,Foamposite One Snakeskin,84,1,84.000000
3,Nike Air Foamposite One Snakeskin,Nike Air Foamposite One,82,32,79.156250
4,Nike Air Foamposite One Snakeskin,Air Foamposite One Snakeskin,92,1,92.000000
...,...,...,...,...,...
495,adidas NMD R1 Red Black Speckle,NMD R1 Red Black Speckle,87,1,87.000000
496,Nike Tailwind Team Orange Tour Yellow,Nike Tailwind Team,65,1,65.000000
497,Nike Tailwind Team Orange Tour Yellow,Tailwind Team Orange,70,1,70.000000
498,Nike Tailwind Team Orange Tour Yellow,Team Orange Tour,60,1,60.000000


In [17]:
df = pd.merge(df, silhouette, how='left', on='name')

In [18]:
df.head()

,code_style,name,brand,date,retail_price,colorway,story,kof_wants,avg_resale_stockx,silhouette,...,nsw,retro,se,pe,gs,hs,ngram,score,total,avg_score
0,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,Air Foamposite One,...,0,0,0,0,0,0,Nike Air Foamposite,73.0,59.0,69.711864
1,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,Air Foamposite One,...,0,0,0,0,0,0,Air Foamposite One,71.0,38.0,67.394737
2,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,Air Foamposite One,...,0,0,0,0,0,0,Foamposite One Snakeskin,84.0,1.0,84.000000
3,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,Air Foamposite One,...,0,0,0,0,0,0,Nike Air Foamposite One,82.0,32.0,79.156250
4,314996-101,Nike Air Foamposite One Snakeskin,Nike,2019-04-15,220,Sail/Black-Habanero Red-Black,"Equipped with an eye-catching upper, this Air ...",2281,305,Air Foamposite One,...,0,0,0,0,0,0,Air Foamposite One Snakeskin,92.0,1.0,92.000000


## Model Preparation

### Identifying target variables

In [19]:
def create_targets(a, b):
    if a==b:
        target = 1
    else:
        target = 0
    return target

In [20]:
df['target'] = df.apply(lambda x: create_targets(x['silhouette'], x['ngram']), axis=1)

### Encode brand

In [21]:
df['brand_code'] = df.groupby(['brand']).ngroup()

## Split test-train

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
data = df[~df.isnull()]

2020-01-28 23:13:39,299 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


In [24]:
X = data[['brand_code', 'score', 'total', 'avg_score']]

In [25]:
y=pd.Series(data.target)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [33]:
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

## LogReg Model

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [31]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.97


In [36]:
pd.DataFrame(y_pred)

,0
0,0
1,0
2,0
3,0
4,0
...,...
18692,0
18693,0
18694,0
18695,0


In [45]:
results = pd.concat([X_test, pd.DataFrame(y_test), pd.DataFrame(y_pred)], axis=1)

In [49]:
results[results['target']==1]

,brand_code,score,total,avg_score,target,0
167,1.0,47.0,238.0,51.268908,1.0,0.0
290,6.0,48.0,14.0,50.714286,1.0,0.0
346,1.0,62.0,9.0,66.000000,1.0,0.0
597,1.0,59.0,46.0,63.608696,1.0,0.0
612,6.0,50.0,93.0,46.322581,1.0,0.0
...,...,...,...,...,...,...
62190,1.0,79.0,58.0,64.068966,1.0,NaN
62203,1.0,67.0,47.0,64.361702,1.0,NaN
62222,1.0,71.0,18.0,68.388889,1.0,NaN
62238,1.0,55.0,47.0,62.170213,1.0,NaN
